In [1]:
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_columns', 500)

In [2]:
leap_df = pd.read_csv("datasets/leap_data_001.csv")
emg_df = pd.read_csv("datasets/emg_data_v1.csv")

In [3]:
leap_df.head()

,Unix Time,Timestamp,Palm x,Palm y,Palm z,Wrist x,Wrist y,Wrist z,Thumb Proximal x,Thumb Proximal y,Thumb Proximal z,Thumb Intermediate x,Thumb Intermediate y,Thumb Intermediate z,Thumb Distal x,Thumb Distal y,Thumb Distal z,Thumb Tip x,Thumb Tip y,Thumb Tip z,Index Proximal x,Index Proximal y,Index Proximal z,Index Intermediate x,Index Intermediate y,Index Intermediate z,Index Distal x,Index Distal y,Index Distal z,Index Tip x,Index Tip y,Index Tip z,Middle Proximal x,Middle Proximal y,Middle Proximal z,Middle Intermediate x,Middle Intermediate y,Middle Intermediate z,Middle Distal x,Middle Distal y,Middle Distal z,Middle Tip x,Middle Tip y,Middle Tip z,Ring Proximal x,Ring Proximal y,Ring Proximal z,Ring Intermediate x,Ring Intermediate y,Ring Intermediate z,Ring Distal x,Ring Distal y,Ring Distal z,Ring Tip x,Ring Tip y,Ring Tip z,Pinky Proximal x,Pinky Proximal y,Pinky Proximal z,Pinky Intermediate x,Pinky Intermediate y,Pinky Intermediate z,Pinky Distal x,Pinky Distal y,Pinky Distal z,Pinky Tip x,Pinky Tip y,Pinky Tip z
0,1.561069e+09,1.561069e+15,53.966988,148.497543,84.236992,44.092537,-11.826508,15.329117,32.816284,-17.849213,39.115044,-7.700520,-4.460510,49.599419,-35.444817,4.608963,56.584419,-54.457291,10.868591,61.458855,-27.160671,11.812378,9.818542,-58.868332,24.661255,-6.298439,-77.143066,16.417236,-13.422653,-84.756599,3.553970,-15.086433,-18.418987,10.669708,-7.552521,-50.664906,23.127060,-32.154343,-71.881516,13.801987,-41.613686,-81.632553,0.468155,-42.486969,-5.527153,6.013611,-22.197262,-31.926264,17.709259,-48.901978,-52.090591,10.510880,-60.573200,-63.343086,-1.257736,-62.892952,6.396278,-2.333298,-34.173100,-12.525803,0.970078,-58.664719,-25.007887,-1.346375,-70.294624,-36.777519,-6.257996,-78.514763
1,1.561069e+09,1.561069e+15,53.935043,148.405991,84.178368,44.033386,-11.873871,15.461998,32.681427,-17.868637,39.218956,-7.868088,-4.452667,49.540733,-35.642456,4.626099,56.392906,-54.687771,10.879211,61.146019,-27.173590,11.852524,9.734039,-58.838871,24.680725,-6.482422,-77.092003,16.421097,-13.643852,-84.698669,3.554321,-15.311935,-18.383682,10.688812,-7.611267,-50.551685,23.157562,-32.309132,-71.756516,13.868851,-41.830467,-81.539734,0.561050,-42.740097,-5.455925,6.007965,-22.216400,-31.769571,17.721420,-48.997555,-51.912022,10.546692,-60.721016,-63.176857,-1.204391,-63.069550,6.491093,-2.360947,-34.153320,-12.479721,0.734467,-58.634399,-25.149132,-2.199631,-69.917137,-37.006935,-7.937576,-77.446251
2,1.561069e+09,1.561069e+15,53.795914,148.030182,83.783249,43.889786,-12.179611,15.631538,32.381157,-18.243423,39.295410,-8.186592,-4.713959,49.395020,-35.978706,4.436554,56.077698,-55.039932,10.734818,60.705765,-27.164951,11.938171,9.653145,-58.705818,24.955734,-6.654617,-76.953636,16.796005,-13.942993,-84.603577,3.968384,-15.711678,-18.288635,10.822479,-7.651268,-50.286522,23.543137,-32.441475,-71.489868,14.418777,-42.123680,-81.352478,1.179886,-43.171078,-5.306950,6.154083,-22.212486,-31.435385,18.123611,-49.061672,-51.553581,11.119614,-60.929134,-62.874023,-0.550339,-63.411057,6.661175,-2.211502,-34.130566,-12.110699,1.140793,-58.730881,-24.956938,-2.408493,-69.630417,-36.850510,-9.115829,-76.246567
3,1.561069e+09,1.561069e+15,52.943245,146.396423,82.245468,43.393517,-13.718826,15.741592,31.555878,-19.699104,39.264008,-8.638901,-5.012253,49.237289,-36.181179,4.930115,55.822418,-55.066471,11.775284,60.394806,-26.841455,12.662048,9.635689,-57.909966,26.761032,-6.679726,-76.350845,19.222412,-14.148811,-84.376175,6.654160,-16.106056,-17.896725,11.474030,-7.628632,-49.377235,25.393448,-32.437126,-70.795403,17.048248,-42.346725,-81.080605,4.156219,-43.629871,-4.975765,6.575653,-22.168301,-30.632776,19.543503,-49.008495,-50.887009,13.288132,-61.061329,-62.563599,2.026520,-63.769379,6.801189,-2.017365,-34.114971,-11.612328,2.689194,-58.764259,-24.835018,-1.139862,-69.104568,-36.889660,-8.689438,-74.387398
4,1.561069e+09,1.561069e+15,52.338261,145.899033,80.235924,43.239796,-14.260208,15.683350,31.323536,-20.2732

In [3]:
emg_df.columns = ['ts', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8']

In [6]:
def crop_dfs(leap_df, emg_df):
    """Crops both DataFrames to start at the same time"""
    i = 0
    while (emg_df['ts'][i] < leap_df['Unix Time'][0]):
        i += 1
    i -= 1
    j = len(emg_df) - 1
    while (emg_df['ts'][j] > leap_df['Unix Time'][len(leap_df) - 1]):
        j -= 1
    j += 1
    return (leap_df, emg_df[i:j].reset_index(drop=True))

In [7]:
leap_df, emg_df = crop_dfs(leap_df, emg_df)
rows = len(emg_df)
columns = 74
length_leap = len(leap_df)
combined = np.zeros((rows, columns))

In [8]:
%%time
j = 0
k = 0
i = 0
while (i < rows):
    while (leap_df["Unix Time"][j] - emg_df["ts"][i] > 0.1):
        print("j {}, i {}, diff {}".format(j, i, leap_df["Unix Time"][j] - emg_df["ts"][i]))
        i += 1
    leap = leap_df.iloc[j].values
    count = 1
    j += 1
    while (j < length_leap and leap_df["Unix Time"][j] <= emg_df["ts"][i]):
        leap += leap_df.iloc[j].values
        j += 1
        count += 1
    leap /= count
    time_leap = leap[0]
    leap = leap[4:]
    emg = emg_df.iloc[i].values
    time_diff = time_leap - emg[0]
    combined[k] = np.concatenate([np.array([time_leap]), np.array([time_diff]), emg, leap])
    k += 1
    i += 1
combined = combined[:k]

j 60226, i 26496, diff 0.5518929958343506
j 60226, i 26497, diff 0.5443940162658691
j 60226, i 26498, diff 0.521888017654419
j 60226, i 26499, diff 0.4996190071105957
j 60226, i 26500, diff 0.47690486907958984
j 60226, i 26501, diff 0.4618809223175049
j 60226, i 26502, diff 0.4393808841705322
j 60226, i 26503, diff 0.42398786544799805
j 60226, i 26504, diff 0.40187788009643555
j 60226, i 26505, diff 0.37183189392089844
j 60226, i 26506, diff 0.3643920421600342
j 60226, i 26507, diff 0.34186601638793945
j 60226, i 26508, diff 0.3194010257720947
j 60226, i 26509, diff 0.29643893241882324
j 60226, i 26510, diff 0.27434587478637695
j 60226, i 26511, diff 0.2593648433685303
j 60226, i 26512, diff 0.2450859546661377
j 60226, i 26513, diff 0.22191095352172852
j 60226, i 26514, diff 0.1994161605834961
j 60226, i 26515, diff 0.18463587760925293
j 60226, i 26516, diff 0.16186094284057617
j 60226, i 26517, diff 0.13890814781188965
j 60226, i 26518, diff 0.12465906143188477
j 92496, i 40722, diff 

j 134702, i 60719, diff 17.47628378868103
j 134702, i 60720, diff 17.45510506629944
j 134702, i 60721, diff 17.440211057662964
j 134702, i 60722, diff 17.417680978775024
j 134702, i 60723, diff 17.39520502090454
j 134702, i 60724, diff 17.378883838653564
j 134702, i 60725, diff 17.357865810394287
j 134702, i 60726, diff 17.335216760635376
j 134702, i 60727, diff 17.32026195526123
j 134702, i 60728, diff 17.28204894065857
j 134702, i 60729, diff 17.27526092529297
j 134702, i 60730, diff 17.26024889945984
j 134702, i 60731, diff 17.237815856933594
j 134702, i 60732, diff 17.215260982513428
j 134702, i 60733, diff 17.200369119644165
j 134702, i 60734, diff 17.17778491973877
j 134702, i 60735, diff 17.155286073684692
j 134702, i 60736, diff 17.140204906463623
j 134702, i 60737, diff 17.117633819580078
j 134702, i 60738, diff 17.0950870513916
j 134702, i 60739, diff 17.08010697364807
j 134702, i 60740, diff 17.05765175819397
j 134702, i 60741, diff 17.026925802230835
j 134702, i 60742, diff

j 135251, i 61833, diff 3.789512872695923
j 135251, i 61834, diff 3.775169849395752
j 135251, i 61835, diff 3.7528278827667236
j 135251, i 61836, diff 3.7362208366394043
j 135251, i 61837, diff 3.7152159214019775
j 135251, i 61838, diff 3.6927430629730225
j 135251, i 61839, diff 3.677690029144287
j 135251, i 61840, diff 3.6551661491394043
j 135251, i 61841, diff 3.6328721046447754
j 135251, i 61842, diff 3.610175132751465
j 135251, i 61843, diff 3.586927890777588
j 135251, i 61844, diff 3.5726470947265625
j 135251, i 61845, diff 3.5500071048736572
j 135251, i 61846, diff 3.5351369380950928
j 135251, i 61847, diff 3.5126490592956543
j 135251, i 61848, diff 3.481982946395874
j 135251, i 61849, diff 3.4595770835876465
j 135251, i 61850, diff 3.444535970687866
j 135251, i 61851, diff 3.4303860664367676
j 135251, i 61852, diff 3.407039165496826
j 135251, i 61853, diff 3.384472131729126
j 135251, i 61854, diff 3.37007212638855
j 135251, i 61855, diff 3.3536529541015625
j 135251, i 61856, dif

In [9]:
combined.shape

(90644, 74)

In [13]:
column_names = ['Leap timestamp', 'timestamp diff', 'emg timestamp',
                'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8',
                'Wrist x', 'Wrist y', 'Wrist z', 'Thumb Proximal x', 'Thumb Proximal y',
                'Thumb Proximal z', 'Thumb Intermediate x', 'Thumb Intermediate y',
                'Thumb Intermediate z', 'Thumb Distal x', 'Thumb Distal y',
                'Thumb Distal z', 'Thumb Tip x', 'Thumb Tip y', 'Thumb Tip z',
                'Index Proximal x', 'Index Proximal y', 'Index Proximal z',
                'Index Intermediate x', 'Index Intermediate y', 'Index Intermediate z',
                'Index Distal x', 'Index Distal y', 'Index Distal z', 'Index Tip x',
                'Index Tip y', 'Index Tip z', 'Middle Proximal x', 'Middle Proximal y',
                'Middle Proximal z', 'Middle Intermediate x', 'Middle Intermediate y',
                'Middle Intermediate z', 'Middle Distal x', 'Middle Distal y',
                'Middle Distal z', 'Middle Tip x', 'Middle Tip y', 'Middle Tip z',
                'Ring Proximal x', 'Ring Proximal y', 'Ring Proximal z',
                'Ring Intermediate x', 'Ring Intermediate y', 'Ring Intermediate z',
                'Ring Distal x', 'Ring Distal y', 'Ring Distal z', 'Ring Tip x',
                'Ring Tip y', 'Ring Tip z', 'Pinky Proximal x', 'Pinky Proximal y',
                'Pinky Proximal z', 'Pinky Intermediate x', 'Pinky Intermediate y',
                'Pinky Intermediate z', 'Pinky Distal x', 'Pinky Distal y',
                'Pinky Distal z', 'Pinky Tip x', 'Pinky Tip y', 'Pinky Tip z']

In [14]:
combined_df = pd.DataFrame(data=combined, columns=column_names)

In [15]:
combined_df.head()

,Leap timestamp,timestamp diff,emg timestamp,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,Wrist x,Wrist y,Wrist z,Thumb Proximal x,Thumb Proximal y,Thumb Proximal z,Thumb Intermediate x,Thumb Intermediate y,Thumb Intermediate z,Thumb Distal x,Thumb Distal y,Thumb Distal z,Thumb Tip x,Thumb Tip y,Thumb Tip z,Index Proximal x,Index Proximal y,Index Proximal z,Index Intermediate x,Index Intermediate y,Index Intermediate z,Index Distal x,Index Distal y,Index Distal z,Index Tip x,Index Tip y,Index Tip z,Middle Proximal x,Middle Proximal y,Middle Proximal z,Middle Intermediate x,Middle Intermediate y,Middle Intermediate z,Middle Distal x,Middle Distal y,Middle Distal z,Middle Tip x,Middle Tip y,Middle Tip z,Ring Proximal x,Ring Proximal y,Ring Proximal z,Ring Intermediate x,Ring Intermediate y,Ring Intermediate z,Ring Distal x,Ring Distal y,Ring Distal z,Ring Tip x,Ring Tip y,Ring Tip z,Pinky Proximal x,Pinky Proximal y,Pinky Proximal z,Pinky Intermediate x,Pinky Intermediate y,Pinky Intermediate z,Pinky Distal x,Pinky Distal y,Pinky Distal z,Pinky Tip x,Pinky Tip y,Pinky Tip z
0,1.561069e+09,0.021962,1.561069e+09,20.0,26.0,45.0,62.0,270.0,38.0,23.0,24.0,44.092537,-11.826508,15.329117,32.816284,-17.849213,39.115044,-7.700520,-4.460510,49.599419,-35.444817,4.608963,56.584419,-54.457291,10.868591,61.458855,-27.160671,11.812378,9.818542,-58.868332,24.661255,-6.298439,-77.143066,16.417236,-13.422653,-84.756599,3.553970,-15.086433,-18.418987,10.669708,-7.552521,-50.664906,23.127060,-32.154343,-71.881516,13.801987,-41.613686,-81.632553,0.468155,-42.486969,-5.527153,6.013611,-22.197262,-31.926264,17.709259,-48.901978,-52.090591,10.510880,-60.573200,-63.343086,-1.257736,-62.892952,6.396278,-2.333298,-34.173100,-12.525803,0.970078,-58.664719,-25.007887,-1.346375,-70.294624,-36.777519,-6.257996,-78.514763
1,1.561069e+09,0.006170,1.561069e+09,20.0,26.0,46.0,69.0,274.0,37.0,24.0,24.0,44.033386,-11.873871,15.461998,32.681427,-17.868637,39.218956,-7.868088,-4.452667,49.540733,-35.642456,4.626099,56.392906,-54.687771,10.879211,61.146019,-27.173590,11.852524,9.734039,-58.838871,24.680725,-6.482422,-77.092003,16.421097,-13.643852,-84.698669,3.554321,-15.311935,-18.383682,10.688812,-7.611267,-50.551685,23.157562,-32.309132,-71.756516,13.868851,-41.830467,-81.539734,0.561050,-42.740097,-5.455925,6.007965,-22.216400,-31.769571,17.721420,-48.997555,-51.912022,10.546692,-60.721016,-63.176857,-1.204391,-63.069550,6.491093,-2.360947,-34.153320,-12.479721,0.734467,-58.634399,-25.149132,-2.199631,-69.917137,-37.006935,-7.937576,-77.446251
2,1.561069e+09,-0.008751,1.561069e+09,20.0,24.0,39.0,67.0,275.0,36.0,20.0,23.0,43.889786,-12.179611,15.631538,32.381157,-18.243423,39.295410,-8.186592,-4.713959,49.395020,-35.978706,4.436554,56.077698,-55.039932,10.734818,60.705765,-27.164951,11.938171,9.653145,-58.705818,24.955734,-6.654617,-76.953636,16.796005,-13.942993,-84.603577,3.968384,-15.711678,-18.288635,10.822479,-7.651268,-50.286522,23.543137,-32.441475,-71.489868,14.418777,-42.123680,-81.352478,1.179886,-43.171078,-5.306950,6.154083,-22.212486,-31.435385,18.123611,-49.061672,-51.553581,11.119614,-60.929134,-62.874023,-0.550339,-63.411057,6.661175,-2.211502,-34.130566,-12.110699,1.140793,-58.730881,-24.956938,-2.408493,-69.630417,-36.850510,-9.115829,-76.246567
3,1.561069e+09,-0.008863,1.561069e+09,21.0,24.0,38.0,69.0,298.0,43.0,20.0,24.0,43.316656,-13.989517,15.712471,31.439707,-19.986164,39.210838,-8.706076,-5.141907,49.146923,-36.223530,4.900223,55.683292,-55.089439,11.816055,60.228619,-26.780442,12.766251,9.667107,-57.800222,27.029053,-6.597591,-76.276329,19.596748,-14.085293,-84.368996,7.078911,-16.085609,-17.835115,11.589149,-7.597649,-49.262276,25.699928,-32.364819,-70.720425,17.503860,-42.311058,-81.090042,4.686867,-43.658333,-4.933538,6.672813,-22.148434,-30.562969,19.782204,-48.945763,-50.844833,13.670326,-61.024803,-62.594429,2.502068,-63.799461,6.804319,-1.940689,-34.118710,-11.554880,3.092934,-58.741636,-24.902935,-0.930557,-68.838737,-36.931690,-8.838234,-73.59

In [20]:
print(combined_df.describe()['timestamp diff'])

count    90644.000000
mean        -0.009675
std          0.004368
min         -0.036651
25%         -0.012138
50%         -0.010131
75%         -0.007594
max          0.094352
Name: timestamp diff, dtype: float64


In [ ]:
combined_df.to_csv("datasets/data_{}_{}.csv".format(len(combined_df), time.time()), index=False)